# TPS Feb 2021
Starter Notebook

## Deleverables
1. EDA
    - What's going on?
    - Show me the data...
2. Model
    - Baseline...
    - Simple...
    - Evaluation...
    - Improvement...
3. RAPIDS Bonus
    - Apply RAPIDS ([Starter Notebook](https://www.kaggle.com/tunguz/tps-feb-2021-rapids-starter))
    - Replace pandas with cuDF & sklearn with cuML
    
    
#### Troubleshooting
- [Data](https://www.kaggle.com/c/tabular-playground-series-feb-2021/data)
- [Overview](https://www.kaggle.com/c/tabular-playground-series-feb-2021/overview)
- [RF Starter Notebook](https://www.kaggle.com/warobson/tps-feb-2021-rf-starter)
- [ML repo on GitHub](https://github.com/gumdropsteve/intro_to_machine_learning)
- [Most simple RAPIDS Notebook submission](https://www.kaggle.com/warobson/simple-rapids-live) (Has stuff like `train_test_split()` with cuml..)
    
#### Load Data

In [ ]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# RAPIDS Bonus [We solve assignment in both ways Rapids/Sklearn]

# RAPIDS Random Forest Model


In [ ]:
import cudf

train = cudf.read_csv("/kaggle/input/tabular-playground-series-feb-2021/train.csv")
test = cudf.read_csv("/kaggle/input/tabular-playground-series-feb-2021/test.csv")
sample_submission = cudf.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')

In [ ]:
train.tail(3)

In [ ]:
test.tail(3)

In [ ]:
sample_submission.tail(3)

### Notes
1. To use sklearn with cudf... convert `cudf.Series` / `cudf.DataFrame` `.to_pandas()`..
2. Using cudf is the same as using pandas but `cudf` instead of `pd`..

In [ ]:
type(train)

In [ ]:
type(train.to_pandas())

In [ ]:
train.to_pandas()

In [ ]:
train[['id', 'cat0', 'cat8', 'cont7']].to_pandas()

In [ ]:
type(train.to_pandas())

In [ ]:
y = train.target

y.to_pandas()

In [ ]:
type(y.to_pandas())

# EDA

In [ ]:
train.shape

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
sns.pairplot(train.to_pandas().sample(100), hue='target');

In [ ]:
train.to_pandas().corr().style.background_gradient(cmap='Blues')

In [ ]:
sns.displot(train.to_pandas().target).set(title='Distripution of Target');

# Prepar Data to modling


## Change Float Type to Float32

In [ ]:
train.columns

In [ ]:
col_name = train[['cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5',
       'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12',
       'cont13', 'target']]

for i in col_name:
    train[i] = train[i].astype(np.float32)

In [ ]:
train.info()

## Encode Data

In [ ]:
train_encode = cudf.get_dummies(train)

test_encode = cudf.get_dummies(test)

In [ ]:
train_encode.info()

In [ ]:
train_encode.columns

In [ ]:
# convert encode data to float32
col_name = ['id','cat0_A', 'cat0_B', 'cat1_A', 'cat1_B', 'cat2_A', 'cat2_B',
       'cat3_A', 'cat3_B', 'cat3_C', 'cat3_D', 'cat4_A', 'cat4_B', 'cat4_C',
       'cat4_D', 'cat5_A', 'cat5_B', 'cat5_C', 'cat5_D', 'cat6_A', 'cat6_B',
       'cat6_C', 'cat6_D', 'cat6_E', 'cat6_G', 'cat6_H', 'cat6_I', 'cat7_A',
       'cat7_B', 'cat7_C', 'cat7_D', 'cat7_E', 'cat7_F', 'cat7_G', 'cat7_I',
       'cat8_A', 'cat8_B', 'cat8_C', 'cat8_D', 'cat8_E', 'cat8_F', 'cat8_G',
       'cat9_A', 'cat9_B', 'cat9_C', 'cat9_D', 'cat9_E', 'cat9_F', 'cat9_G',
       'cat9_H', 'cat9_I', 'cat9_J', 'cat9_K', 'cat9_L', 'cat9_M', 'cat9_N',
       'cat9_O']

for i in col_name:
    train_encode[i] = train_encode[i].astype(np.float32)

## Split Data

In [ ]:
from cuml.preprocessing import train_test_split

X = train_encode.drop('target', axis=1)
y = train_encode.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

## Scale Data

### Here we will use two methods:

* First: We will scale continuos variables only 

* Second: We do not use scaling at all

## Using the First method: Scaling continuos variables
[Why to not scale categorical variables](https://stats.stackexchange.com/questions/169350/centering-and-scaling-dummy-variables/169358)

In [ ]:





from cuml.experimental.preprocessing import StandardScaler
# scale the X data ONLY
scaler = StandardScaler()
# better save it to new var 
X_train_features_scale=X_train.copy()
X_test_features_scale=X_test.copy()


X_train_features_scale[['cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6','cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13']]=scaler.fit_transform(X_train_features_scale[['cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6','cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13']])
X_test_features_scale[['cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6','cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13']]=scaler.transform(X_test_features_scale[['cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6','cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13']])


X_train_scale = X_train_features_scale # Scale have to be after split data & on train only, .fit_transform() is only for train data
X_test_scale  = X_test_features_scale      # Scale have to be after split data & on test only,  .transform() is only for test data

# Simple Model [Random Forest Regressor]

In [ ]:
from cuml.ensemble import RandomForestRegressor

rfr1 = RandomForestRegressor()

rfr1.fit(X_train_scale, y_train)

In [ ]:
rfr1_preds = rfr1.predict(X_test_scale)

rfr1_preds.tail()

## Simple Model Evaluation

In [ ]:
from cuml.metrics import mean_squared_error

cuml_model_1_mse = mean_squared_error(y_true=y_test,
                   y_pred=rfr1_preds,
                   squared=False)
cuml_model_1_mse

## Using the Second Method: Not Scaling
[When could you use scale?](https://medium.com/greyatom/why-how-and-when-to-scale-your-features-4b30ab09db5e)

# Simple Model [Random Forest Regressor]

In [ ]:

rfr2 = RandomForestRegressor()

rfr2.fit(X_train, y_train)

In [ ]:
rfr2_preds = rfr2.predict(X_test)

rfr2_preds.tail()

## Simple Model Evaluation

In [ ]:
from cuml.metrics import mean_squared_error

cuml_model_2_mse = mean_squared_error(y_true=y_test,
                   y_pred=rfr2_preds,
                   squared=False)
cuml_model_2_mse

# Baseline Model

In [ ]:
def baseline_model(n_preds, pred):
    # just predict the average
    return cudf.Series([pred for n in range(n_preds)])

# make baseline preds
baseline_preds = baseline_model(len(y_test), np.mean(y_train))

In [ ]:
# change preds type to float32
baseline_preds = baseline_preds.astype(np.float32)

In [ ]:
from cuml.metrics import mean_squared_error

baseline_mse=mean_squared_error(y_true=y_test,
                   y_pred=baseline_preds,
                   squared=False)
baseline_mse

# Improvement Model [Random Forest Regressor]

 
# Dask GridSearchCV on GPU

### Note: try to do it but failed 


In [ ]:
# download 'dask-ml'
!pip -q install 'dask-ml'

In [ ]:
import dask_ml.model_selection as dcv

[Reference](https://ml.dask.org/modules/generated/dask_ml.model_selection.GridSearchCV.html)


In [ ]:



dask_parameters = {
    'n_estimators': [100, 200, 500],
    'max_depth' : [5, 10 , 16, 20, 50],
    'split_criterion' :[2, 3]
    }


rfr_grid = dcv.GridSearchCV(
                    rfr2,
                    dask_parameters,cv=7
                    )

Tried to optimize the model but an error keep showing!

In [ ]:
# show Error: AttributeError: 'NoneType' object has no attribute 'fit'
# rfr_grid.fit(X_train_scale,y_train)

# Sklearn Random Forest Model


# Improvement Model [Random Forest Regressor]

## We will use gridsearch to find best parameters for the model.
### But first Need to have Sklearn model 

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/test.csv")
sample_submission = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')

In [ ]:
train.head()

## We will examin two methods:
* First: one-hot encoding using Ordinal Encoder
* Second: one-hot encoding using Get Dummies

## First: Ordinal Encoder

In [ ]:
# Encode data
import category_encoders as ce

# Ordinally Encoded DF 
encoder = ce.OrdinalEncoder()
train_encode = encoder.fit_transform(train)

# Ordinally Encoded DF 
encoder = ce.OrdinalEncoder()
test_encode = encoder.fit_transform(test)

## Spliting the dataset

In [ ]:
# Split Data
from sklearn.model_selection import train_test_split

X = train_encode.drop('target', axis=1)
y = train_encode.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
train_encode.head()

In [ ]:
# Random forest model
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor()

rfr.fit(X_train, y_train)

In [ ]:
preds = rfr.predict(X_test)

preds[-5: ]

In [ ]:
# MSE for model
from sklearn.metrics import mean_squared_error

skl_model_1_mse = mean_squared_error(y_test, preds,squared=False)
skl_model_1_mse

In [ ]:
# MAE for model
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_test, preds)

### Model Optimazition

In [ ]:

# grid search with sklearn
from sklearn.model_selection import GridSearchCV

param_grid = { 
    'n_estimators': [100, 200, 500],
    'bootstrap': [True],
    'max_depth' : [5, 10 , 16, 20, 50]
}


grid = GridSearchCV(rfr,param_grid,cv=3)
grid


In [ ]:
# grid.fit(X_train, y_train)

In [ ]:
# best_c=grid.best_params_
# best_c

In [ ]:

# ## Change Model Parameters
# improved_rfr_c = RandomForestRegressor(n_estimators=best_c['n_estimators'],
#                                       bootstrap=best_c['bootstrap'],
#                                       max_depth =best_c['max_depth'])

# improved_rfr_c.fit(X_train, y_train)


In [ ]:
# improved_preds_c = improved_rfr_c.predict(X_test)

In [ ]:

# # MSE for model
# from sklearn.metrics import mean_squared_error

# sk_rf_c=mean_squared_error(y_test, improved_preds_c,squared=False)
# sk_rf_c


## Second: Get Dummies

In [ ]:
train_dum=pd.get_dummies(train)
test_dum=pd.get_dummies(test)

In [ ]:
train_dum.head()

In [ ]:
X_d = train_dum.drop('target', axis=1)
y_d = train_dum.target

dX_train, dX_test, dy_train, dy_test = train_test_split(X_d, y_d, test_size=0.2)

In [ ]:
# Random forest model
from sklearn.ensemble import RandomForestRegressor

rfr_d = RandomForestRegressor()

rfr_d.fit(dX_train, dy_train)

In [ ]:
preds_d = rfr_d.predict(dX_test)

preds_d[-5: ]

In [ ]:
# MSE for model
from sklearn.metrics import mean_squared_error

skl_model_d_mse = mean_squared_error(dy_test, preds_d,squared=False)
skl_model_d_mse

In [ ]:
# MAE for model
from sklearn.metrics import mean_absolute_error

skl_model_d_mae=mean_absolute_error(dy_test, preds_d)
skl_model_d_mae

Model Optimization


In [ ]:


# grid search with sklearn
from sklearn.model_selection import GridSearchCV

param_grid = { 
    'n_estimators': [100, 200, 500],
    'bootstrap': [True],
    'max_depth' : [5, 10 , 16, 20, 50]
}


grid_d = GridSearchCV(rfr_d,param_grid,cv=3)
grid_d

In [ ]:
# grid_d.fit(dX_train, dy_train)

In [ ]:
# best_d=grid_d.best_params_
# best_d

In [ ]:
# grid_d.best_score_

In [ ]:
# ## Change Model Parameters
# improved_rfr_d = RandomForestRegressor(n_estimators=best_d['n_estimators'],
#                                      bootstrap=best_d['bootstrap'],
#                                      max_depth =best_d['max_depth'])

# improved_rfr_d.fit(dX_train, dy_train)

In [ ]:
# improved_preds_d = improved_rfr_d.predict(dX_test)

In [ ]:
# improved_d_mse=mean_squared_error(dy_test, improved_preds_d,squared=False)
# improved_d_mse

# The Best Model

In [ ]:


best_score= {'Baseline Model':baseline_mse,
                 'RAPIDS Random Forest Regressor Model with Scaled Variables':cuml_model_1_mse,
                 'RAPIDS Random Forest Regressor Model with No Scaled Variables':cuml_model_2_mse,
                 'SKlearn Random Forest Regressor Model with Ordinal Encoder':skl_model_1_mse,
                # 'SKlearn Random Forest Regressor Model with Ordinal Encoder After Optimization':sk_rf_c,
                 'SKlearn Random Forest Regressor Model with Get Dummies':skl_model_d_mse#,
                # 'SKlearn Random Forest Regressor Model with Get Dummies After Optimization':improved_d_mse
                 
                 }



In [ ]:
min_key = min(best_score, key=best_score.get)
min_value =best_score[min_key]

In [ ]:


print(f'As seen in the above scores tests, we select {min_key} with a root mean squared error score:{min_value}')



In [ ]:
# Best Model
models= {'rfr1':cuml_model_1_mse,
                 'rfr2':cuml_model_2_mse,
                 'rfr':skl_model_1_mse,
                 #'improved_rfr_c':sk_rf_c,
                 'rfr_d':skl_model_d_mse#,
                 #'improved_rfr_d':improved_d_mse
                 
                 }



In [ ]:
best_model_name = min(models, key=models.get)
best_model=models[best_model_name]

In [ ]:

models_names_conventer= {'rfr1':rfr1,
                 'rfr2':rfr2,
                 'rfr':rfr,
                 #'improved_rfr_c':improved_rfr_c,
                 'rfr_d':rfr_d#,
                 #'improved_rfr_d':improved_rfr_d
                 
                 }

# submit to kaggle

In [ ]:
x=models_names_conventer[best_model_name]

In [ ]:
ordinal_encoder=['rfr','improved_rfr_c']
get_dum=['rfr1','rfr2','rfr_d','improved_rfr_d']

In [ ]:
if best_model_name in ordinal_encoder:
    test=test_encode

    
elif best_model_name in get_dum:
    test=test_dum
    test['cat6_G']=0

In [ ]:

the_best_predict=x.predict(test)
sample_submission['target'] = the_best_predict

sample_submission

In [ ]:
sample_submission.to_csv('submission.csv', index=False)